## Factorization benchmarks
#### ..and other experiments

Consider a multivariate polynomial $F$ over the finite field $\mathbb{F}_q$ in the variables $x_1,\ldots, x_n$.
Throughout this page $D$ will stand for the total degree of $F$, $T$ for the number of non- zero terms of $F$, and $n$ for the number of variables. 
We will also write $L$ for the number of field operations in $\mathbb{F}_q$ required to evaluate $F$.

Measuring the complexity of factoring $F$ into irreducibles is a challenging task due to the large number of parameters involved. To name a few, the timings depend on 

1. The sparsity of $F$ and the factors of $F$, and the ratio between $T$, $D$, and $n$. In this report, we are generally concerned with sparse polynomials such that $n$ is proportional to or surpasses $D$, which often happens in practice.
2. The ground field $\mathbb{F}_q$, namely, the size of $q$. Here, we treat the case when the arithmetic in $\mathbb{F}_q$ can be implemented efficiently, i.e., when $q$ is a prime small enough to fit in a 64-bit register.

At the same time, it can be beneficial to draw the benchmark problems from the "real-world" tasks in computer algebra and other applications. We consider the testsuite consisting of the following problems:

##### 1. $F$ is a determinant
$F$ is the determinant of a matrix over a ring of polynomials.
One example would be the determinant of the matrix $A$ defined by
$$
A = 
\begin{pmatrix}
x_1 & x_2 & \cdots & x_n \\
x_{n+1} & x_{n+2} & \cdots & x_{2n} \\
\vdots & \vdots & \ddots & \vdots \\
x_{(n-1)n+1} & x_{(n-1)n+2} & \cdots & x_{n^2}
\end{pmatrix}
$$
The determinant of $A$ is an *irreducible* polynomial in $n^2$ variables, and the number of non-zero terms is $n!$.

The matrix can have structure that would adjust the properties of the determinant. Examples are:
- **Hankel matrix**. The determinant is an irreducible polynomial of degree $D$ in $2n$ variables. The number of terms is around $n$ times less than in the general case.
- **Toeplitz matrix**. The determinant is a polynomial of degree $D$ in $n$ variables. Almost the same number of terms as for the Hankel matrix, but splits into $2$ factors.
- **Circlulant matrix**. The determinant is a polynomial of degree $D$ in $n$ variables. It splits into $\mathcal{O}(n)$ factors.
- **Vandermonde matrix**. The determinant is a polynomial of degree $\frac{n^2 - n}{2}$ in $n$ variables that splits into the product of $\frac{n^2 - n}{2}$ linear factors. 

One might also want to dehomogenize the homogeneous determinants before factoring them,

##### 2. $F$ is a product of random polynomials

In this case, $F$ can be fine-tuned to have desired number and sparsity of its factors.


##### 3. $F$ is a resultant (perhaps, of resultants)



Our algorithm may make decisions at various steps of the execution that directly affect the overall performance.

##### Recursive vs. revealing approaches. 

Recursive approach finds a **true factor** of $F$ and descends recursively to the quotient of the factor and $F$. In practice, several true factors that depend on one of the indeterminates are found at once. Revealing approach artificially creates an indeterminate that reveals a complete factorization of $F$ and finds all of the factors in one go.

Examples of transformations that would reveal a complete factorization are:

1. 1
2. 2

##### Factor normalization.

One approach is to apply a linear shift to the variables to create a constant term in the original polynomial.

Another approach is to apply a power-product substitution to the variables to assure that there is only one trailing term of the lowest total degree.
To this end, we apply a unimodular transform to variables that increases the total degree in all variables but the main one by a constant factor.
We want the transform that is easy to generate, and increases the total degree $D$ by no more than a constant factor of $2$ or $3$.

1. Generate an id. matrix and gradually fill the lower/upper part with random small elements.
2. Generate an id. matrix and gradually fill the lower/upper diagonal with random small elements.
3. Generate an id. matrix and multiply by a random combination of upper/lower triangular unimodular matrices.
4. 

Under the transformation, $F$ will have a single trailing term $A_\ell$ of the lowest total degree. We divide all of the evaluated factors by the trailing coefficient and multiply by the term $A_\ell$ to get the normalized factors.

##### Selecting the main variable.

Out of all $n$ variable $x_1,\ldots, x_n$, we select one to be the main variable.

Conditions on the main variable $x_i$ are:

- The leading coefficient of $F$ in $x_i$ does not vanish at $(0, \ldots, 0)$,
- $F(0,\ldots, x_i, \ldots, 0)$ is squarefree.

Hopes on the main variable $x_i$ are:

- $x_i$ has a small degree in $F$,
- $x_i$ reveals many factors in $F$ *(in recursive approach)*,
- there is a single trailing term of the lowest total degree w.r.t. $x_i$ *(in power-product approach)*.

----

In [1]:
using AbstractAlgebra, Nemo
using LinearAlgebra
include("../src/ExactSparseInterpolations.jl")

Main.ExactSparseInterpolations

In [266]:
#=
    Generate a random monic **irreducible** polynomial over F_q
    with n variables, partial degrees up to d, and exactly t terms

    Each monomial x1^d1...xn^dn is sampled from a uniform distribution
        di ~ U(0, d)
    Each coefficient Cj is sampled from a uniform distribution
        Cj ~ U(1, q - 1)
=#
function random_irreducible_polynomial(F_q, n::Integer, t::Integer, d::Integer)
    R, xs = PolynomialRing(F_q, ["x$i" for i in 1:n])
    q = Int(characteristic(F_q))
    f = nothing
    while true
        monomials = Vector{elem_type(R)}(undef, 0)
        coeffs = Vector{elem_type(F_q)}(undef, 0)
        while length(monomials) < t
            monomial = prod([xs[i]^rand(0:d) for i in 1:n])
            if monomial ∉ monomials
                push!(monomials, monomial)
                push!(coeffs, rand(F_q, 1:q - 1))
            end
        end
        f = sum([coeffs[i]*monomials[i] for i in 1:t])
        length(factor(f)) == 1 && break
    end
    divexact(f, leading_coefficient(f))
end

random_irreducible_polynomial (generic function with 1 method)

In [267]:
function generate_resultant(K, n::Int)
    x = gen(K)
    y = gen(K)
    f = prod(x - rand(K) for i in 1:n)
    g = prod(y - rand(K) for i in 1:n)
    return resultant(f, g)    
end

generate_resultant (generic function with 1 method)

----

In [268]:
# Generating matrices
function generate_determinant_general(K, n::Int)
    R, xs = PolynomialRing(K, ["x$i" for i in 1:n^2])
    S = MatrixSpace(R, n, n)
    M = zero(S)
    for i in 1:n, j in 1:n
        M[i, j] = xs[j + (i-1)*n]
    end
    det(M)
end

function generate_determinant_vandermonde(K, n::Int)
    R, xs = PolynomialRing(K, ["x$i" for i in 1:n])
    S = MatrixSpace(R, n, n)
    M = zero(S)
    for i in 1:n
        for j in 1:n
            M[i, j] = xs[i]^(j - 1)
        end
    end
    det(M)
end

function generate_determinant_hankel(K, n::Int)
    R, xs = PolynomialRing(K, ["x$i" for i in 1:2n])
    S = MatrixSpace(R, n, n)
    M = zero(S)
    for i in 1:n
        for j in 1:n
            M[i, j] = xs[i + j - 1]
        end
    end
    det(M)
end

function generate_determinant_toeplitz(K, n::Int)
    # note this is a special case of toeplitz matrices
    # with n variables instead of 2n variables
    R, xs = PolynomialRing(K, ["x$i" for i in 1:n])
    S = MatrixSpace(R, n, n)
    M = zero(S)
    for i in 1:2n-1
        if i <= n
            for j in 1:n - i + 1
                M[j, j + i - 1] = xs[i]
            end
        else
            for j in (i - n + 1):n
                M[j, j - (i - n)] = xs[i - n + 1]
            end
        end
    end
    det(M)
end

function generate_determinant_circulant(K, n::Int)
    R, xs = PolynomialRing(K, ["x$i" for i in 1:n])
    S = MatrixSpace(R, n, n)
    M = zero(S)
    for i in 1:2n-1
        if i <= n
            for j in 1:n - i + 1
                M[j, j + i - 1] = xs[i]
            end
        else
            for j in (i - n + 1):n
                M[j, j - (i - n)] = xs[2n - i + 1]
            end
        end
    end
    det(M)
end

generate_determinant_circulant (generic function with 1 method)

In [269]:
f1 = generate_determinant_general(K, 6);
f2 = generate_determinant_hankel(K, 6);
f3 = generate_determinant_toeplitz(K, 6);
f4 = generate_determinant_circulant(K, 6);
f5 = generate_determinant_vandermonde(K, 6);

In [270]:
length(f1), length(f2), length(f3), length(f4), length(f5)

(720, 231, 120, 68, 720)

In [271]:
total_degree(f1), total_degree(f2), total_degree(f3), total_degree(f4), total_degree(f5)

(6, 6, 6, 6, 15)

In [272]:
length(factor(f1)), length(factor(f2)), length(factor(f3)), length(factor(f4)), length(factor(f5))

(1, 1, 2, 6, 15)

In [352]:
using AbstractAlgebra, Nemo
using LinearAlgebra
include("../src/ExactSparseInterpolations.jl")

f = generate_determinant_toeplitz(K, 4)
R = parent(f)
xs = vars(f)
f = evaluate(f, vcat(xs[1:end-1], [R(1)]))

fact1 = factor(f)
length(fact1)

2

In [319]:
collect(keys(fact1.fac))

2-element Vector{gfp_mpoly}:
 x1^2 + 2147483646*x1*x2 + 2147483646*x1 + 2147483646*x2^2 + 2*x2*x3 + x2 + 2147483646*x3^2
 x1^2 + x1*x2 + x1 + 2147483646*x2^2 + 2147483645*x2*x3 + x2 + 2147483646*x3^2

In [320]:
ExactSparseInterpolations.top_level_factorize(
    f,
    benchmark=true,
    strategy=:recursive,
    mainvar=:smalldegree
)

┌ Info: Factoring prim
│   F = x1^4 + 2147483644*x1^2*x2^2 + 2147483645*x1^2*x3^2 + 2147483646*x1^2 + 4*x1*x2^2*x3 + 4*x1*x2*x3 + x2^4 + 2147483645*x2^3 + 2147483645*x2^2*x3^2 + x2^2 + 2147483645*x2*x3^2 + x3^4
└ @ Main.ExactSparseInterpolations c:\data\projects\interpol\ExactSparseInterpolations.jl\src\factorization\multivariate-factor-ff.jl:312
┌ Info: 
│   success = false
│   T = 16
│   Pi = x1^4 + 2147483644*x1^2*x2^2 + 2147483645*x1^2*x3^2 + 2147483646*x1^2 + 4*x1*x2^2*x3 + 4*x1*x2*x3 + x2^4 + 2147483645*x2^3 + 2147483645*x2^2*x3^2 + x2^2 + 2147483645*x2*x3^2 + x3^4
└ @ Main.ExactSparseInterpolations c:\data\projects\interpol\ExactSparseInterpolations.jl\src\factorization\multivariate-factor-ff.jl:318


1-element Vector{gfp_mpoly}:
 x1^4 + 2147483644*x1^2*x2^2 + 2147483645*x1^2*x3^2 + 2147483646*x1^2 + 4*x1*x2^2*x3 + 4*x1*x2*x3 + x2^4 + 2147483645*x2^3 + 2147483645*x2^2*x3^2 + x2^2 + 2147483645*x2*x3^2 + x3^4

In [263]:
function uwu(x; args...)
    println(args)
    @time Dict(args)
    println(keys(args))
    if !all(key -> key ∈ [:a, :b, :c], keys(args))
        @warn "Beda"
    end
end

uwu (generic function with 1 method)

In [264]:
uwu(f, 
benchmark=true,
strategy=:recursive,
mainvar=:smalldegree)

Base.Pairs{Symbol, Any, Tuple{Symbol, Symbol, Symbol}, NamedTuple{(:benchmark, :strategy, :mainvar), Tuple{Bool, Symbol, Symbol}}}(:benchmark => true, :strategy => :recursive, :mainvar => :smalldegree)
  0.000004 seconds (7 allocations: 608 bytes)
(:benchmark, :strategy, :mainvar)


┌ Warning: Beda
└ @ Main c:\data\projects\interpol\ExactSparseInterpolations.jl\perf\factor-benchmarks.ipynb:6


In [252]:
hmm = Base.Pairs{Symbol, Any, Tuple{Symbol, Symbol, Symbol}, NamedTuple{(:benchmark, :strategy, :mainvar), Tuple{Bool, Symbol, Symbol}}}(:benchmark => true, :strategy => :recursive, :mainvar => :smalldegree)

MethodError: MethodError: no method matching Base.Pairs{Symbol, Any, Tuple{Symbol, Symbol, Symbol}, NamedTuple{(:benchmark, :strategy, :mainvar), Tuple{Bool, Symbol, Symbol}}}(::Pair{Symbol, Bool}, ::Pair{Symbol, Symbol}, ::Pair{Symbol, Symbol})
Closest candidates are:
  Base.Pairs{K, V, I, A}(::Any, ::Any) where {K, V, I, A} at essentials.jl:33

In [209]:
@time fact = factor(f)
fact.fac

  0.000860 seconds (11 allocations: 1008 bytes)


Dict{gfp_mpoly, Int64} with 2 entries:
  x1 + x2 + x3 + x4 + x5                                                    => 1
  x1^4 + 2147483646*x1^3*x2 + 2147483646*x1^3*x3 + 2147483646*x1^3*x4 + 21… => 1

In [219]:
@time factorization = ExactSparseInterpolations.top_level_factorize(f)

  0.023249 seconds (11.69 k allocations: 720.559 KiB, 97.48% compilation time)


1-element Vector{gfp_mpoly}:
 x1^5 + 2147483642*x1^3*x2*x5 + 2147483642*x1^3*x3*x4 + 5*x1^2*x2^2*x4 + 5*x1^2*x2*x3^2 + 5*x1^2*x3*x5^2 + 5*x1^2*x4^2*x5 + 2147483642*x1*x2^3*x3 + 5*x1*x2^2*x5^2 + 2147483642*x1*x2*x3*x4*x5 + 2147483642*x1*x2*x4^3 + 2147483642*x1*x3^3*x5 + 5*x1*x3^2*x4^2 + 2147483642*x1*x4*x5^3 + x2^5 + 2147483642*x2^3*x4*x5 + 5*x2^2*x3^2*x5 + 5*x2^2*x3*x4^2 + 2147483642*x2*x3^3*x4 + 2147483642*x2*x3*x5^3 + 5*x2*x4^2*x5^2 + x3^5 + 5*x3^2*x4*x5^2 + 2147483642*x3*x4^3*x5 + x4^5 + x5^5

----

Benchmark for matrices

In [335]:
function benchmark_determinants(K, ns)
    println("================\nGeneral matrix (irreducible):")
    for n in ns
        f = generate_determinant_general(K, n)
        @time "Nemo" fact1 = factor(f)
        @time "Ours" fact2 = ExactSparseInterpolations.top_level_factorize(
            f,
            benchmark=true
        )
        @assert prod(fact2) == f
        @assert length(fact1) == length(fact2)
        bench = ExactSparseInterpolations.dump_benchmarks()
    end 
    # println("================\nToeplitz matrix (2+ factors):")
    # for n in ns
    #     f = generate_determinant_toeplitz(K, n)
    #     @time "Nemo" fact1 = factor(f)
    #     @time "Ours" fact2 = ExactSparseInterpolations.top_level_factorize(
    #         f,
    #         benchmark=true
    #     )
    #     @assert prod(fact2) == f
    #     @assert length(fact1) == length(fact2)
    #     bench = ExactSparseInterpolations.dump_benchmarks()
    # end 
    # println("================\nCirculant matrix (n factors):")
    # for n in ns
    #     f = generate_determinant_circulant(K, n)
    #     @time "Nemo" fact1 = factor(f)
    #     @time "Ours" fact2 = ExactSparseInterpolations.top_level_factorize(
    #         f,
    #         benchmark=true
    #     )
    #     @assert prod(fact2) == f
    #     @assert length(fact1) == length(fact2)
    #     bench = ExactSparseInterpolations.dump_benchmarks()
    # end 
    println("================\nVandermonde matrix (n^2 factors):")
    for n in ns
        f = generate_determinant_vandermonde(K, n)
        @time "Nemo" fact1 = factor(f)
        @time "Ours" fact2 = ExactSparseInterpolations.top_level_factorize(
            f,
            benchmark=true
        )
        # @assert length(fact1) == length(fact2)
        # @assert prod(fact2) == f
        bench = ExactSparseInterpolations.dump_benchmarks()
    end 
end

benchmark_determinants (generic function with 1 method)

In [353]:
f = generate_determinant_vandermonde(K, 4);

In [354]:
fact1.fac

Dict{gfp_mpoly, Int64} with 2 entries:
  x1^2 + 2147483646*x1*x2 + 2147483646*x1 + 2147483646*x2^2 + 2*x2*x3 + x2… => 1
  x1^2 + x1*x2 + x1 + 2147483646*x2^2 + 2147483645*x2*x3 + x2 + 2147483646… => 1

In [355]:
fact2

1-element Vector{gfp_mpoly}:
 x1^3*x2^2*x3 + 2147483646*x1^3*x2^2*x4 + 2147483646*x1^3*x2*x3^2 + x1^3*x2*x4^2 + x1^3*x3^2*x4 + 2147483646*x1^3*x3*x4^2 + 2147483646*x1^2*x2^3*x3 + x1^2*x2^3*x4 + x1^2*x2*x3^3 + 2147483646*x1^2*x2*x4^3 + 2147483646*x1^2*x3^3*x4 + x1^2*x3*x4^3 + x1*x2^3*x3^2 + 2147483646*x1*x2^3*x4^2 + 2147483646*x1*x2^2*x3^3 + x1*x2^2*x4^3 + x1*x3^3*x4^2 + 2147483646*x1*x3^2*x4^3 + 2147483646*x2^3*x3^2*x4 + x2^3*x3*x4^2 + x2^2*x3^3*x4 + 2147483646*x2^2*x3*x4^3 + 2147483646*x2*x3^3*x4^2 + x2*x3^2*x4^3

In [349]:
fact1 = factor(f)
fact2 = ExactSparseInterpolations.top_level_factorize(
    f,
    benchmark=true
);
bench = ExactSparseInterpolations.dump_benchmarks()

Dict{Symbol, Any} with 15 entries:
  :t_total                      => [0.0002679]
  :t_finding_power_product      => Float64[]
  :t_selecting_main_variable    => Float64[]
  :t_removing_content           => Float64[]
  :t_factoring_content          => [7.0e-6]
  :v_points_used                => Int64[]
  :t_select_main_variable       => [8.43e-5, 5.7e-5, 4.11e-5, 3.91e-5]
  :t_interpolating_coefficients => Float64[]
  :v_transform_degrees          => Tuple{Int64, Int64}[]
  :t_prim_and_sqfree            => Float64[]
  :v_transform_matrices         => Tuple{Matrix{Int64}, Matrix{Int64}}[]
  :t_first_bivariate_factor     => Float64[]
  :t_evaluating_coefficients    => Float64[]
  :v_main_var                   => NamedTuple{(:main_var_idx, :degrees), Tuple{…
  :t_many_hensel_liftings       => Float64[]

In [345]:
sum(bench[:t_removing_content])

0.2705279000000001

In [336]:
K = Nemo.GF(2^31-1)
ns = 3, 4, 5, 6, 7, 8
benchmark_determinants(K, ns)

General matrix (irreducible):
Nemo: 0.000235 seconds (9 allocations: 880 bytes)
Ours: 0.000330 seconds (523 allocations: 40.578 KiB)
Nemo: 0.000972 seconds (9 allocations: 944 bytes)
Ours: 0.001164 seconds (1.15 k allocations: 97.688 KiB)
Nemo: 0.003623 seconds (9 allocations: 1008 bytes)
Ours: 0.005452 seconds (2.24 k allocations: 222.141 KiB)
Nemo: 0.044408 seconds (9 allocations: 1.094 KiB)


Ours: 0.046697 seconds (4.49 k allocations: 652.625 KiB)


Nemo: 0.454816 seconds (9 allocations: 1.172 KiB)


Ours: 0.544331 seconds (11.24 k allocations: 2.850 MiB)


Nemo: 5.573186 seconds (9 allocations: 1.297 KiB)


Ours: 7.905760 seconds (49.94 k allocations: 23.370 MiB)
Vandermonde matrix (n^2 factors):


Nemo: 0.006475 seconds (13 allocations: 1.094 KiB)
Ours: 0.001130 seconds (394 allocations: 27.609 KiB)
Nemo: 0.000377 seconds (19 allocations: 1.609 KiB)
Ours: 0.001420 seconds (895 allocations: 65.203 KiB)
Nemo: 0.001442 seconds (27 allocations: 2.234 KiB)
Ours: 0.000977 seconds (1.93 k allocations: 149.672 KiB)
Nemo: 0.004431 seconds (51 allocations: 5.703 KiB)
Ours: 0.019102 seconds (5.74 k allocations: 543.203 KiB)


Nemo: 0.037381 seconds (63 allocations: 6.734 KiB)
Ours: 0.060576 seconds (33.14 k allocations: 3.418 MiB)


Nemo: 0.099291 seconds (77 allocations: 8.547 KiB)


Ours: 0.315546 seconds (303.92 k allocations: 36.857 MiB)
